### Customizing what happens in ``.fit()``, `.compile` and `.evaluate()`.

In this notebook we are going to use subclassing to create our custom `.fit()`, `.compile()` and `.evaluate()` methods on our model.

The implementation is the same throughout all models achitectures:
1. Sequantial API
2. Functional API
3. Subclassing API

In this notebook we will use the `Sequential` API but late on we will use all these apis.

* [Docs](https://keras.io/guides/customizing_what_happens_in_fit/)

### Imports

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import datasets
import numpy as np

### Configuring the ``device`` for the environment.


In [2]:
physical_devices = tf.config.list_physical_devices("GPU")
tf.config.experimental.set_memory_growth(physical_devices[0], True)

### Let's create a model that will train on the `MNIST` dataset.

In [3]:
(X_train, y_train), (X_test, y_test) = datasets.mnist.load_data()
X_train.shape

11493376/11490434 [==============================] - 0s 0us/step


(60000, 28, 28)

In [4]:
def normalize(image):
  image = tf.convert_to_tensor(image.astype('float32'))/255
  return image

In [5]:
X_train_tensors =tf.convert_to_tensor(list(map(normalize, X_train)))
X_test_tensors = tf.convert_to_tensor(list(map(normalize, X_test)))

y_test_tensors = tf.convert_to_tensor(y_test)
y_train_tensors = tf.convert_to_tensor(y_train)

In [ ]:
y_test_tensors[:2]

<tf.Tensor: shape=(2,), dtype=uint8, numpy=array([7, 2], dtype=uint8)>

 ### Let's create a `Sequential` model that will fit on our custom `.fit()`, `.compile()` and `.evaluate()` method.

The compile method will be a simple one. It takes 3 keword args, metrics, loss, optimizer
```
```

 [2nd way](https://keras.io/guides/customizing_what_happens_in_fit/)

In [54]:

class CustomCompileFitEvaluate(keras.Model):
  def __init__(self, model):
    super(CustomCompileFitEvaluate, self).__init__()
    self.model = model

  # .compile() -> compiling the model
  def compile(self, loss, optimizer, metrics):
    super().compile()
    self.loss = loss
    self.optimizer = optimizer
    self.custom_metrics = metrics

  # .fit() -> trainning the model
  def train_step(self, data):
    x, y = data
    # forward pass
    with tf.GradientTape() as tape:
      y_pred = self.model(x, training=True)
      # loss
      loss = self.loss(y, y_pred)
    # Calculate gradients
    gradients = tape.gradient(loss, self.trainable_variables)
    # updating the weights
    self.optimizer.apply_gradients(zip(gradients, self.trainable_variables))
    # updating the metrics
    self.custom_metrics.update_state(y, y_pred)

    return {"loss": loss ,"accuracy": self.custom_metrics.result()}
  # .evaluate() -> Evaluating the model
  def test_step(self, data):
    x, y = data
    y_pred = self.model(x, training=False)
    # loss
    loss = self.loss(y, y_pred)
    self.custom_metrics.update_state(y, y_pred)
    return {"loss": loss ,"accuracy": self.custom_metrics.result()}

### Fitting the model using our custom:

```
compile() -> fit() -> evaluate()
```

In [56]:
model = keras.Sequential([
      keras.layers.Flatten(),
      keras.layers.Dense(64, activation="relu"),
      keras.layers.Dense(128, activation="relu"),
      keras.layers.Dense(10, activation="softmax")
])

new_model = CustomCompileFitEvaluate(model)
new_model.compile(loss=keras.losses.SparseCategoricalCrossentropy(),
                  optimizer=keras.optimizers.Adam(learning_rate=0.001),
                  metrics = keras.metrics.SparseCategoricalAccuracy()
              )
new_model.fit(X_train_tensors, y_train_tensors, batch_size=32, epochs=5,
              validation_data=(X_test_tensors, y_test))


Epoch 1/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.2722 - accuracy: 0.8606 - val_loss: 0.0575 - val_accuracy: 0.9590
Epoch 2/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.1193 - accuracy: 0.9621 - val_loss: 0.0135 - val_accuracy: 0.9644
Epoch 3/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0865 - accuracy: 0.9729 - val_loss: 0.0131 - val_accuracy: 0.9714
Epoch 4/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0674 - accuracy: 0.9791 - val_loss: 0.0139 - val_accuracy: 0.9746
Epoch 5/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0578 - accuracy: 0.9834 - val_loss: 0.1781 - val_accuracy: 0.9743


### Evaluating the `model`

In [57]:
new_model.evaluate(X_test_tensors, y_test_tensors, batch_size=32, verbose=1)

313/313 [==============================] - 1s 2ms/step - loss: 0.0810 - accuracy: 0.9723


[0.9743000268936157, 0.17808373272418976]

> That's all about creating our own custom `fit()`, `compile()` and `evaluate()` to go in depth [read](https://keras.io/guides/customizing_what_happens_in_fit/)